<a href="https://colab.research.google.com/github/bsjeon87/DeepReinforcementLearning/blob/master/AdaptiveBitrate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글 drive mount 방법
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Munch
!pip install yacs
!pip install -U transformers==4.45.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.2
    Uninstalling transformers-4.50.2:
      Successfully uninstalled transformers-4.50.2


tflean 에러 발생 -> !pip install tflearn

experience data 수집

# command examples:
    # python generate_exp_pool.py --models mpc --traces fcc-train --video video1 --trace-num -1 --seed 1 --fixed-order --cuda-id 0


In [ ]:
!python /content/drive/MyDrive/netllm/adaptive_bitrate_streaming/generate_exp_pool.py --models mpc --trace fcc-train --video video1 --trace-num -1 --seed 1 --fixed-order --cuda-id 0

2025-03-25 03:33:16.387019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742873596.407168    2876 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742873596.413299    2876 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 03:33:16.433473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Namespace(models=['mpc'], trace='fcc-train', video='video1', trace_num=-1, seed=1, cuda_id=0, fixed_order=True)
/cont

    # >>> for debug <<<
    # args.exp_pool_path = 'artifacts/exp_pools/exp_pool.pkl'
    # args.plm_type = 'llama'
    # args.plm_size = 'base'
    # args.rank = 128
    # args.state_feature_dim = 256
    # args.num_epochs = 1
    # args.eval_per_epoch = 1
    # args.adapt = True
    # args.test = True
    # args.device = 'cuda:0'
    # args.device_out = 'cuda:0'
    # args.which_layer = -1
    # args.seed = 100003
    # >>> for debug <<<

python run_plm.py --adapt --test --grad-accum-steps 32 --seed 666 --plm-type llama --plm-size base --rank 128 --device cuda:0 --state-feature-dim 256 --w 20 --gamma 1. --lr 0.0001 --warmup-steps 2000 --num-epochs 80 --eval-per-epoch 2 --target-return-scale 1

In [ ]:
!python /content/drive/MyDrive/netllm/adaptive_bitrate_streaming/run_plm.py --adapt --test --grad-accum-steps 32 --seed 666 --plm-type llama --plm-size base --rank 128 --device cuda:0 --state-feature-dim 256 --w 20 --gamma 1. --lr 0.0001 --warmup-steps 2000 --num-epochs 80 --eval-per-epoch 2 --target-return-scale 1

Arguments:
Namespace(exp_pool_path='/content/drive/MyDrive/netllm/adaptive_bitrate_streaming/artifacts/exp_pools/exp_pool.pkl', sample_step=None, trace='fcc-test', trace_num=100, video='video1', fixed_order=False, plm_type='llama', plm_size='base', rank=128, state_feature_dim=256, w=20, gamma=1.0, lr=0.0001, weight_decay=0.0001, warmup_steps=2000, num_epochs=80, eval_per_epoch=2, save_checkpoint_per_epoch=2, target_return_scale=1.0, which_layer=-1, adapt=True, test=True, grad_accum_steps=32, seed=666, scale=1000, model_dir=None, device='cuda:0', device_out='cuda:0', device_mid=None)
Loading traces from /content/drive/MyDrive/netllm/adaptive_bitrate_streaming/data/traces/test/fcc-test/
Experience dataset info:
Munch({'max_reward': np.float64(4.3), 'min_reward': np.float64(-85.0127377757031), 'max_return': 0.04661987504979122, 'min_return': 0.0006304750495579298, 'min_timestep': 0, 'max_timestep': 46, 'max_action': 5, 'min_action': 0})
If tokenizer is loaded:  [128000, 15339, 1917] 

pad

In [ ]:
!pip show transformers

Name: transformers
Version: 4.49.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
!python /content/drive/MyDrive/netllm/adaptive_bitrate_streaming/run_plm.py --test --grad-accum-steps 32 --seed 666 --plm-type llama --plm-size base --rank 128 --device cuda:0 --state-feature-dim 256 --w 20 --gamma 1. --lr 0.0001 --warmup-steps 2000 --num-epochs 80 --eval-per-epoch 2 --target-return-scale 1

Arguments:
Namespace(exp_pool_path='/content/drive/MyDrive/netllm/adaptive_bitrate_streaming/artifacts/exp_pools/exp_pool.pkl', sample_step=None, trace='fcc-test', trace_num=100, video='video1', fixed_order=False, plm_type='llama', plm_size='base', rank=128, state_feature_dim=256, w=20, gamma=1.0, lr=0.0001, weight_decay=0.0001, warmup_steps=2000, num_epochs=80, eval_per_epoch=2, save_checkpoint_per_epoch=2, target_return_scale=1.0, which_layer=-1, adapt=False, test=True, grad_accum_steps=32, seed=666, scale=1000, model_dir=None, device='cuda:0', device_out='cuda:0', device_mid=None)
Loading traces from /content/drive/MyDrive/netllm/adaptive_bitrate_streaming/data/traces/test/fcc-test/
Experience dataset info:
Munch({'max_reward': np.float64(4.3), 'min_reward': np.float64(-85.0127377757031), 'max_return': 0.04661987504979122, 'min_return': 0.0006304750495579298, 'min_timestep': 0, 'max_timestep': 46, 'max_action': 5, 'min_action': 0})
If tokenizer is loaded:  [128000, 15339, 1917] 

pa